<a href="https://colab.research.google.com/github/soralh1611/Gen-AI-Agents/blob/main/Soral_UW_Fraud_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# The '!' allows you to run shell commands directly from the notebook
!pip install Faker

In [4]:
import json
import random
from faker import Faker

fake = Faker()
records = []

# Possible fraud markers we want our agent to learn
RISK_LEVELS = ["low", "medium", "high"]
FLAG_REASONS = [
    "none",
    "synthetic_identity_match",
    "ip_velocity_high",
    "ssn_randomization_mismatch",
    "disposable_email_domain"
]

for i in range(1000):
    # Standard profile data
    is_fraud = random.random() < 0.15  # 15% chance of being a fraud record

    record = {
        "id": f"app_{i:04d}",
        "name": fake.name(),
        "email": fake.email() if not is_fraud else f"user{random.randint(100,999)}@dispostable.com",
        "ssn_last4": f"{random.randint(1000, 9999)}",
        "ip_address": fake.ipv4_public(),
        "risk_level": "high" if is_fraud else random.choice(["low", "medium"]),
        "flag_reason": random.choice(FLAG_REASONS[1:]) if is_fraud else "none",
        "monthly_income": random.randint(2000, 15000),
        "is_synthetic_suspect": is_fraud
    }
    records.append(record)

# Save as JSONL (Required format for Vertex AI Data Stores)
with open("underwriting_data_1000.jsonl", "w") as f:
    for entry in records:
        f.write(json.dumps(entry) + "\n")

print("Created 1000 records in underwriting_data_1000.jsonl")

Created 1000 records in underwriting_data_1000.jsonl


In [5]:
from google.colab import files

# This will trigger your browser to download the file directly from the cloud
files.download('underwriting_data_1000.jsonl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.adk.agents import LlmAgent
from google.adk.tools import agent_tool
from google.adk.tools.google_search_tool import GoogleSearchTool
from google.adk.tools import url_context
from google.adk.tools import VertexAiSearchTool

identity_verification_expert_google_search_agent = LlmAgent(
  name='Identity_Verification_Expert_google_search_agent',
  model='gemini-2.5-flash',
  description=(
      'Agent specialized in performing Google searches.'
  ),
  sub_agents=[],
  instruction='Use the GoogleSearchTool to find information on the web.',
  tools=[
    GoogleSearchTool()
  ],
)
identity_verification_expert_url_context_agent = LlmAgent(
  name='Identity_Verification_Expert_url_context_agent',
  model='gemini-2.5-flash',
  description=(
      'Agent specialized in fetching content from URLs.'
  ),
  sub_agents=[],
  instruction='Use the UrlContextTool to retrieve content from provided URLs.',
  tools=[
    url_context
  ],
)
identity_verification_expert = LlmAgent(
  name='identity_verification_expert',
  model='gemini-2.5-flash',
  description=(
      'Helps identify synthetic identity fraud'
  ),
  sub_agents=[],
  instruction='',
  tools=[
    agent_tool.AgentTool(agent=identity_verification_expert_google_search_agent),
    agent_tool.AgentTool(agent=identity_verification_expert_url_context_agent)
  ],
)
soral_fraud_detection_agent__google_search_agent = LlmAgent(
  name='Soral_Fraud_Detection_Agent__google_search_agent',
  model='gemini-2.5-flash',
  description=(
      'Agent specialized in performing Google searches.'
  ),
  sub_agents=[],
  instruction='Use the GoogleSearchTool to find information on the web.',
  tools=[
    GoogleSearchTool()
  ],
)
soral_fraud_detection_agent__url_context_agent = LlmAgent(
  name='Soral_Fraud_Detection_Agent__url_context_agent',
  model='gemini-2.5-flash',
  description=(
      'Agent specialized in fetching content from URLs.'
  ),
  sub_agents=[],
  instruction='Use the UrlContextTool to retrieve content from provided URLs.',
  tools=[
    url_context
  ],
)
soral_fraud_detection_agent__vertex_ai_search_agent = LlmAgent(
  name='Soral_Fraud_Detection_Agent__vertex_ai_search_agent',
  model='gemini-2.5-flash',
  description=(
      'Agent specialized in performing Vertex AI Search.'
  ),
  sub_agents=[],
  instruction='Use the VertexAISearchTool to find information using Vertex AI Search.',
  tools=[
    VertexAiSearchTool(
      data_store_id='projects/soral-vertex-a/locations/us/collections/default_collection/dataStores/soral-fraud-agent-data_1767313129128'
    )
  ],
)
root_agent = LlmAgent(
  name='Soral_Fraud_Detection_Agent_',
  model='gemini-2.5-flash',
  description=(
      'Identify potential fraud in loan applications by analyzing IP metadata, verifying identity documents, and cross-referencing applicant data against internal records.'
  ),
  sub_agents=[identity_verification_expert],
  instruction='Playbook Instructions: Identity Verification Specialist\n\n1. GREET the user professionally and ask for the Application ID or specific identity details they wish to verify.\n\n2. SEARCH the ${DATA_STORE: soral-fraud-agent-data_1767313129128} using the provided Application ID to retrieve the associated risk profile.\n\n3. ANALYZE the following fields from the database record: risk_level, flag_reason, and is_synthetic_suspect.\n\n4. IF the database returns a risk_level of \"high\" or is_synthetic_suspect is true: DO NOT approve the application.\n\n5. INFORM the user that their application has been flagged for a Manual Security Review due to data inconsistencies.\n\n6. STRICTLY AVOID revealing internal technical flag names like \"synthetic_identity_match\" to the user.\n\n7. IF the database returns a risk_level of \"low\": VERIFY if the user\'s current session IP address matches the known_ips list in the record.\n\n8. IF they match, proceed with a \"Low Risk\" confirmation.\n\n9. IF there is an IP mismatch, ask the user to confirm their current location for \"Step-up\" verification.\n\n10. IF no record is found in the database, ask the user to double-check the ID provided and NEVER invent or \"hallucinate\" a result.',
  tools=[
    agent_tool.AgentTool(agent=soral_fraud_detection_agent__google_search_agent),
    agent_tool.AgentTool(agent=soral_fraud_detection_agent__url_context_agent),
    agent_tool.AgentTool(agent=soral_fraud_detection_agent__vertex_ai_search_agent)
  ],
)